<a href="https://colab.research.google.com/github/Alsr96/Neural-networks/blob/master/neural_network_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow

There are errors in the following codes to highlight important points.

In [0]:
import tensorflow as tf
import numpy as np

## Basics

### Placeholders

* Placeholders: undefined variables, not initialised. Represents some value, holds a place for some value. Useful to define inputs and outputs of a function.
* A plceholder can be defined using datatype, shape of the value and name.

In [31]:
a = tf.placeholder(tf.float32, shape=(4,), name='placeholder_a')
b = tf.placeholder(tf.float32, shape=(4,), name='placeholder_b')
c = tf.placeholder(tf.float32, shape=(None,4), name='placeholder_c')
d = tf.placeholder(tf.float32)
e = tf.placeholder()

TypeError: ignored

* We defined a bunch of placeholders in the above code.
* Running the above code will give an error, dtype is a necessary argument, shape and name are not.
* Why is shape not necessary? We are not giving it any values, we might not know what it might be. None represents that any size can go theough it.
* Name represents the variable's name on the graph which may not be the same as the assigned variable to the value.
* They don't have values but we can use them in functions.
* Note: no spaces in name.

In [32]:
add_a_b = tf.add(a,b,name='addition')
multiply_c_d = tf.multiply(c,d, name='multiplication')
matmul_c_d = tf.matmul(c,d, name='matrix_multiplication')
matmul_b_c = tf.matmul(b,c, name='matrix_multiplication')

ValueError: ignored

* line 4 gives us an error, the dimensions must match
* add and multiply are from tf.math module which perform elementwise, they can also be accessed with tf.math.add and tf.math.multiply respectively
* matmul performs a matrix multiplication, from tf.linalg (linear algebra) module. Can be accessed as tf.linalg.matmul, it has other arguments to transpose, find adjoint etc of the inputs.
* We have defined the functions but they won't give us any values since the placeholders don't have any values.

To pass values through them we need to create a session and 'feed' the values.

In [33]:
with tf.Session() as sess:
  print('a = \n',sess.run(a, feed_dict={a:[1,2,3,4]}))
  print('d = \n',sess.run(d, feed_dict={d:[[1,2],
                                           [3,4],
                                           [5,6]]}))
  
  print('a+b = \n',sess.run(add_a_b, feed_dict={a:[1,2,3,4],
                                                b:[10,11,12,13]}))
  
  print('c.*d = \n',sess.run(multiply_c_d, feed_dict={c:[[1,2,3,4],
                                                         [5,6,7,8]],
                                                      d:[[11,12,13,14],
                                                         [15,16,17,18]]}))
  
  print('c*d = \n',sess.run(matmul_c_d, feed_dict={c:[[1,2,3,4],
                                                      [5,6,7,8],
                                                      [9,10,11,12],
                                                      [13,14,15,16]],
                                                   d:[[1],
                                                      [2],
                                                      [3],
                                                      [4]]}))


a = 
 [1. 2. 3. 4.]
d = 
 [[1. 2.]
 [3. 4.]
 [5. 6.]]
a+b = 
 [11. 13. 15. 17.]
c.*d = 
 [[ 11.  24.  39.  56.]
 [ 75.  96. 119. 144.]]
c*d = 
 [[ 30.]
 [ 70.]
 [110.]
 [150.]]


* So we create a session using tf.Session() (note the capital 'S') and use run.
* run has 2 main arguments: the 1st one is what it fetches, the function, placeholder etc we need to run and second one feeds the input.
* Notice that we can pass any shape in 'd' . Data with shapes (3,2), (2,4) and (4,1) were passed.
* We can pass different shapes in 'c' too but it's restricted when compared to 'd'. We can only vary the 1st dimension. Data with shapes (2,4) and (4,4) were passed.

### Variables

* They need to be initialised.
* They store values which change over iterations.
* Defined using tf.Variable() (capital 'v' in variable)

In [0]:
x = tf.Variable(initial_value=[1,2,3,4],name='variable_x')

* The initial value is defined using the 1st argument.
* To return the value of a variable we need to run it in a session.

In [40]:
with tf.Session() as sess:
  print('x: \n',sess.run(x))

FailedPreconditionError: ignored

If we run the session we get an error. We have only defined the initial value, we also need to initialise it which is done using tf.global_variables_initializer()

In [49]:
initialise_variables = tf.global_variables_initializer()
f=tf.placeholder(tf.int32)
with tf.Session() as sess:
  sess.run(initialise_variables)
  print('x = \n',sess.run(x))
  print('x+x = \n',sess.run(x+x))
  print('tf.add(x,x) = \n',sess.run(tf.add(x,x)))
  print('x.*f = \n',sess.run(tf.multiply(x,f),feed_dict={f:[4,5,6,7]}))

x = 
 [1 2 3 4]
x+x = 
 [2 4 6 8]
tf.add(x,x) = 
 [2 4 6 8]
x.*f = 
 [ 4 10 18 28]


Note: Maintain data types in functions.

## Building a Neural network

We now have a basic idea and we can use it to create networks.

### Single layer perceptron

1. It's a linear classifier, can be used to classify linearly separable data.
2. The input vector is multiplied with a weight matrix, a bias vector is added to the result to get an output.
3. This output is compared with a target and the difference is used to train the classifier.

Let's first define the perceptron
1. it has an input vector and an output vector.

In [0]:
# create a perceptron class
class perceptron(object):
  # initialise the network
  def __init__(self, n_inputs, n_outputs, learning_rate):

    self.n_inputs = n_inputs
    self.n_outputs = n_outputs
    self.learning_rate = learning_rate

  def build_perceptron(self):

    self.inputs = tf.placeholder(tf.float32,
                                 shape=(None,self.n_inputs),
                                 name='input')
    self.outputs = tf.placeholder(tf.float32,
                                  shape=(None,self.n_outputs),
                                  name='output')
    self.targets = tf.placeholder(tf.float32,
                                  shape=(None,self.n_outputs),
                                  name='target')
    